In [2]:
# FIRST PART OF ASSIGNMENT 3, TEST ACCURACY OF THE GIVEN ONE-LAYER MLP is 0.97 BASED ON MY CODE

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline


n_input = 784 # e.g. MNIST data input (img shape: 28*28)
n_hidden = 392 # hidden layer num units (e.g. half of input units)
n_classes  = 10 # e.g. MNIST total classes (0-9 digits)

# tf Graph variables
x = tf.placeholder("float", [None, n_input], name='x')
y = tf.placeholder("float", [None, n_classes], name='y')

# Store layers weight & bias
stddev = 0.1 # <== This greatly affects accuracy!! 
weights = {
    'h': tf.Variable(tf.random_normal([n_input, n_hidden], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], stddev=stddev))
}
biases = {
    'b': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Create model
hidden_layer = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['h']), biases['b']))
activation = tf.sigmoid(tf.matmul(hidden_layer, weights['out']) + biases['out']) # prediction of <<sigmoid(netSum)>>

# Define loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(activation, y)) # Softmax loss

# 1. Load MNIST dataset

print ("Download and Extract MNIST dataset")
mnist = input_data.read_data_sets('data/', one_hot=True)
print (" tpye of 'mnist' is ", type(mnist))
print (" number of trian data is %d" % (mnist.train.num_examples))
print (" number of test data is %d" % (mnist.test.num_examples))

# 2. define optimizer using tf.train.AdamOptimizer
# 3. train the model by changing learning_rate, training_epochs, batch_size.

learning_rate = 0.01
noOfiterations = 5
batch_size = 100
display_step = 1

# adam optimizer converges quickly than gradient descent optimizer
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

# open session
with tf.Session() as sess:
    sess.run(init)
    
    # training
    for iteration in range(noOfiterations):
        
        average_cost = 0.
        noOfBatches = int(mnist.train.num_examples / batch_size)
        
        # loop through batches
        for i in range(noOfBatches):
            
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            # fit training using batch
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            
            # calculate average cost
            average_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys}) / noOfBatches
            
            # write logs at every iteration
            #summary_str = sess.run(summary_op, feed_dict={x: batch_xs, y: batch_ys})
            #summary_writer.add_summary(summary_str, iteration * noOfBatches + i)
            
        # display logs per iteration
        if iteration % display_step == 0:
            print "Iteration:", '%4d' % (iteration + 1), "cost=", "{:.5f}".format(average_cost)

    print "\nResults:\n"

    # Test model
    predictions = tf.equal(tf.argmax(activation, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})
    writer = tf.train.SummaryWriter('/tmp/tf_logs/mlp_model',sess.graph)

Download and Extract MNIST dataset
Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
(" tpye of 'mnist' is ", <class 'tensorflow.contrib.learn.python.learn.datasets.mnist.DataSets'>)
 number of trian data is 55000
 number of test data is 10000
Iteration:    1 cost= 1.54193
Iteration:    2 cost= 1.49662
Iteration:    3 cost= 1.48637
Iteration:    4 cost= 1.48138
Iteration:    5 cost= 1.47801

Results:

Accuracy: 0.9733
